## High accuracy Low-Rank Tensor Completion Imputer (HaLRTC-imputer)

This notebook shows how to implement a HaLRTC imputer on some real-world data sets (e.g., PeMS-4W, PeMS-8W).

In [1]:
import numpy as np

def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

def mat2ten(mat, dim, mode):
    index = list()
    index.append(mode)
    for i in range(dim.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, list(dim[index]), order = 'F'), 0, mode)

def svt(mat, tau):
    u, s, v = np.linalg.svd(mat, full_matrices = False)
    vec = s - tau
    vec[vec < 0] = 0
    return np.matmul(np.matmul(u, np.diag(vec)), v)

<div class="alert alert-block alert-warning">
<ul>
<li><b><code>compute_mape</code>:</b> <font color="black">Compute the value of Mean Absolute Percentage Error (MAPE).</font></li>
<li><b><code>compute_rmse</code>:</b> <font color="black">Compute the value of Root Mean Square Error (RMSE).</font></li>
</ul>
</div>

> Note that $$\mathrm{MAPE}=\frac{1}{n} \sum_{i=1}^{n} \frac{\left|y_{i}-\hat{y}_{i}\right|}{y_{i}} \times 100, \quad\mathrm{RMSE}=\sqrt{\frac{1}{n} \sum_{i=1}^{n}\left(y_{i}-\hat{y}_{i}\right)^{2}},$$ where $n$ is the total number of estimated values, and $y_i$ and $\hat{y}_i$ are the actual value and its estimation, respectively.

In [2]:
def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

In [3]:
def HaLRTC_imputer(dense_tensor, sparse_tensor, alpha: list, rho: float, epsilon: float, maxiter: int):
    dim = np.array(sparse_tensor.shape)
    tensor_hat = sparse_tensor
    pos_missing = np.where(sparse_tensor == 0)
    pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    B = [np.zeros(sparse_tensor.shape) for _ in range(len(dim))]
    Y = [np.zeros(sparse_tensor.shape) for _ in range(len(dim))]
    last_ten = sparse_tensor.copy()
    snorm = np.linalg.norm(sparse_tensor)
    
    it = 0
    while True:
        for k in range(len(dim)):
            B[k] = mat2ten(svt(ten2mat(tensor_hat + Y[k] / rho, k), alpha[k] / rho), dim, k)
        tensor_hat[pos_missing] = ((sum(B) - sum(Y) / rho) / 3)[pos_missing]
        for k in range(len(dim)):
            Y[k] = Y[k] - rho * (B[k] - tensor_hat)
        tol = np.linalg.norm((tensor_hat - last_ten)) / snorm
        last_ten = tensor_hat.copy()
        it += 1
        if it % 1 == 0:
            print('Iter: {}'.format(it))
            print('Tolerance: {:.6}'.format(tol))
            print('MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], tensor_hat[pos_test])))
            print('RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], tensor_hat[pos_test])))
            print()
        if (tol < epsilon) or (it >= maxiter):
            break
    
    print('Total iteration: {}'.format(it))
    print('Tolerance: {:.6}'.format(tol))
    print('MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], tensor_hat[pos_test])))
    print('RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], tensor_hat[pos_test])))
    print()
    
    return tensor_hat

### Guangzhou data set

In [4]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

## Random Missing (RM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim2, dim3) + 0.5 - missing_rate)

In [5]:
import time
start = time.time()
alpha = [1/3, 1/3, 1/3]
rho = 1e-4
epsilon = 1e-3
maxiter = 100
tensor_hat = HaLRTC_imputer(dense_tensor, sparse_tensor, alpha, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes'% ((end - start)/60.0))

Iter: 1
Tolerance: 0.407334
MAPE: 0.374693
RMSE: 15.8754

Iter: 2
Tolerance: 0.289718
MAPE: 0.143681
RMSE: 5.49401

Iter: 3
Tolerance: 0.06677
MAPE: 0.205244
RMSE: 7.58703

Iter: 4
Tolerance: 0.0584779
MAPE: 0.128642
RMSE: 4.99906

Iter: 5
Tolerance: 0.0534122
MAPE: 0.08798
RMSE: 3.70426

Iter: 6
Tolerance: 0.0212762
MAPE: 0.0886419
RMSE: 3.69403

Iter: 7
Tolerance: 0.0097951
MAPE: 0.0853956
RMSE: 3.54407

Iter: 8
Tolerance: 0.0100704
MAPE: 0.0820509
RMSE: 3.40066

Iter: 9
Tolerance: 0.00784817
MAPE: 0.0802388
RMSE: 3.32517

Iter: 10
Tolerance: 0.00696585
MAPE: 0.0791065
RMSE: 3.27957

Iter: 11
Tolerance: 0.00634687
MAPE: 0.0786294
RMSE: 3.25854

Iter: 12
Tolerance: 0.00566851
MAPE: 0.0789906
RMSE: 3.26792

Iter: 13
Tolerance: 0.00511002
MAPE: 0.0800156
RMSE: 3.30222

Iter: 14
Tolerance: 0.00449179
MAPE: 0.0813616
RMSE: 3.34984

Iter: 15
Tolerance: 0.00381733
MAPE: 0.0824564
RMSE: 3.38897

Iter: 16
Tolerance: 0.00328528
MAPE: 0.0832855
RMSE: 3.4185

Iter: 17
Tolerance: 0.00281639
MAPE:

In [6]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.7

## Random Missing (RM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim2, dim3) + 0.5 - missing_rate)

In [7]:
import time
start = time.time()
alpha = [1/3, 1/3, 1/3]
rho = 1e-4
epsilon = 1e-3
maxiter = 100
tensor_hat = HaLRTC_imputer(dense_tensor, sparse_tensor, alpha, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes'% ((end - start)/60.0))

Iter: 1
Tolerance: 0.358431
MAPE: 0.753673
RMSE: 31.1454

Iter: 2
Tolerance: 0.500467
MAPE: 0.428133
RMSE: 18.2922

Iter: 3
Tolerance: 0.506762
MAPE: 0.171374
RMSE: 6.7769

Iter: 4
Tolerance: 0.39957
MAPE: 0.232554
RMSE: 8.30408

Iter: 5
Tolerance: 0.211426
MAPE: 0.362583
RMSE: 12.8847

Iter: 6
Tolerance: 0.053698
MAPE: 0.362513
RMSE: 13.0775

Iter: 7
Tolerance: 0.132553
MAPE: 0.270951
RMSE: 9.96395

Iter: 8
Tolerance: 0.182978
MAPE: 0.15506
RMSE: 5.96075

Iter: 9
Tolerance: 0.16804
MAPE: 0.0995607
RMSE: 4.13934

Iter: 10
Tolerance: 0.113738
MAPE: 0.127969
RMSE: 5.14025

Iter: 11
Tolerance: 0.0488108
MAPE: 0.146058
RMSE: 5.87267

Iter: 12
Tolerance: 0.0267523
MAPE: 0.138929
RMSE: 5.58805

Iter: 13
Tolerance: 0.0519741
MAPE: 0.120099
RMSE: 4.81286

Iter: 14
Tolerance: 0.0589828
MAPE: 0.10501
RMSE: 4.22904

Iter: 15
Tolerance: 0.0486344
MAPE: 0.101974
RMSE: 4.10729

Iter: 16
Tolerance: 0.0295344
MAPE: 0.10523
RMSE: 4.19735

Iter: 17
Tolerance: 0.0115944
MAPE: 0.106339
RMSE: 4.23162

Iter

In [8]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.7

## Random Missing (RM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim2, dim3) + 0.5 - missing_rate)

pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
halrtc_ghat = dense_tensor[pos_test] - tensor_hat[pos_test]
np.save('halrtc_ghat.npy', halrtc_ghat)

In [9]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

## Non-random Missing (NM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim3) + 0.5 - missing_rate)[:, None, :]

In [10]:
import time
start = time.time()
alpha = [1/3, 1/3, 1/3]
rho = 1e-4
epsilon = 1e-3
maxiter = 100
tensor_hat = HaLRTC_imputer(dense_tensor, sparse_tensor, alpha, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes'% ((end - start)/60.0))

Iter: 1
Tolerance: 0.235336
MAPE: 0.624224
RMSE: 26.0904

Iter: 2
Tolerance: 0.23388
MAPE: 0.29377
RMSE: 12.7397

Iter: 3
Tolerance: 0.19147
MAPE: 0.147662
RMSE: 5.74807

Iter: 4
Tolerance: 0.101124
MAPE: 0.223238
RMSE: 8.03603

Iter: 5
Tolerance: 0.0361931
MAPE: 0.22068
RMSE: 8.0422

Iter: 6
Tolerance: 0.0487858
MAPE: 0.157949
RMSE: 6.03977

Iter: 7
Tolerance: 0.0501386
MAPE: 0.113637
RMSE: 4.57103

Iter: 8
Tolerance: 0.0344453
MAPE: 0.11059
RMSE: 4.44367

Iter: 9
Tolerance: 0.0173076
MAPE: 0.116341
RMSE: 4.62273

Iter: 10
Tolerance: 0.00947072
MAPE: 0.115409
RMSE: 4.60021

Iter: 11
Tolerance: 0.00990802
MAPE: 0.111299
RMSE: 4.47092

Iter: 12
Tolerance: 0.00930905
MAPE: 0.108118
RMSE: 4.36743

Iter: 13
Tolerance: 0.00724853
MAPE: 0.106688
RMSE: 4.31868

Iter: 14
Tolerance: 0.00514691
MAPE: 0.1064
RMSE: 4.30611

Iter: 15
Tolerance: 0.00386021
MAPE: 0.10662
RMSE: 4.30966

Iter: 16
Tolerance: 0.00315943
MAPE: 0.107031
RMSE: 4.3198

Iter: 17
Tolerance: 0.0026949
MAPE: 0.107464
RMSE: 4.331

In [11]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.7

## Non-random Missing (NM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim3) + 0.5 - missing_rate)[:, None, :]

In [12]:
import time
start = time.time()
alpha = [1/3, 1/3, 1/3]
rho = 1e-4
epsilon = 1e-3
maxiter = 100
tensor_hat = HaLRTC_imputer(dense_tensor, sparse_tensor, alpha, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes'% ((end - start)/60.0))

Iter: 1
Tolerance: 0.214361
MAPE: 0.857836
RMSE: 35.0709

Iter: 2
Tolerance: 0.242574
MAPE: 0.706372
RMSE: 29.6455

Iter: 3
Tolerance: 0.21863
MAPE: 0.570728
RMSE: 24.7628

Iter: 4
Tolerance: 0.194969
MAPE: 0.454222
RMSE: 20.3215

Iter: 5
Tolerance: 0.177932
MAPE: 0.35633
RMSE: 16.2519

Iter: 6
Tolerance: 0.164696
MAPE: 0.275653
RMSE: 12.5909

Iter: 7
Tolerance: 0.151324
MAPE: 0.212475
RMSE: 9.47448

Iter: 8
Tolerance: 0.135868
MAPE: 0.168969
RMSE: 7.14174

Iter: 9
Tolerance: 0.116187
MAPE: 0.147136
RMSE: 5.86696

Iter: 10
Tolerance: 0.0906448
MAPE: 0.14418
RMSE: 5.56511

Iter: 11
Tolerance: 0.0632855
MAPE: 0.148105
RMSE: 5.6458

Iter: 12
Tolerance: 0.0439581
MAPE: 0.14748
RMSE: 5.64246

Iter: 13
Tolerance: 0.0376353
MAPE: 0.140725
RMSE: 5.45702

Iter: 14
Tolerance: 0.0389789
MAPE: 0.132047
RMSE: 5.18352

Iter: 15
Tolerance: 0.0403905
MAPE: 0.125689
RMSE: 4.95736

Iter: 16
Tolerance: 0.0388747
MAPE: 0.12345
RMSE: 4.86095

Iter: 17
Tolerance: 0.0344575
MAPE: 0.124729
RMSE: 4.89004

Iter

### London data set

In [13]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.3

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]
dim1, dim2 = dense_mat.shape
del binary_mat

## Random missing (RM)
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)
dense_tensor = dense_mat.reshape([dim1, 30, 24]).transpose(0, 2, 1)
sparse_tensor = sparse_mat.reshape([dim1, 30, 24]).transpose(0, 2, 1)
del dense_mat, sparse_mat

In [14]:
import time
start = time.time()
alpha = [1/3, 1/3, 1/3]
rho = 1e-4
epsilon = 1e-3
maxiter = 100
tensor_hat = HaLRTC_imputer(dense_tensor, sparse_tensor, alpha, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes'% ((end - start)/60.0))

Iter: 1
Tolerance: 0.221318
MAPE: 0.678996
RMSE: 16.9011

Iter: 2
Tolerance: 0.265461
MAPE: 0.318282
RMSE: 8.39668

Iter: 3
Tolerance: 0.222263
MAPE: 0.118209
RMSE: 3.2345

Iter: 4
Tolerance: 0.124343
MAPE: 0.169536
RMSE: 3.81329

Iter: 5
Tolerance: 0.0681562
MAPE: 0.200074
RMSE: 4.43558

Iter: 6
Tolerance: 0.0464858
MAPE: 0.187822
RMSE: 4.18189

Iter: 7
Tolerance: 0.0425534
MAPE: 0.154985
RMSE: 3.49322

Iter: 8
Tolerance: 0.0387575
MAPE: 0.122971
RMSE: 2.82509

Iter: 9
Tolerance: 0.0308079
MAPE: 0.103373
RMSE: 2.4162

Iter: 10
Tolerance: 0.0218014
MAPE: 0.0949049
RMSE: 2.23751

Iter: 11
Tolerance: 0.0144591
MAPE: 0.0918592
RMSE: 2.17148

Iter: 12
Tolerance: 0.00942602
MAPE: 0.0907766
RMSE: 2.14658

Iter: 13
Tolerance: 0.00669855
MAPE: 0.0902998
RMSE: 2.13493

Iter: 14
Tolerance: 0.0055952
MAPE: 0.0899796
RMSE: 2.12754

Iter: 15
Tolerance: 0.00521283
MAPE: 0.0897159
RMSE: 2.12235

Iter: 16
Tolerance: 0.00497488
MAPE: 0.0895074
RMSE: 2.1192

Iter: 17
Tolerance: 0.00462767
MAPE: 0.089370

In [15]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.7

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]
dim1, dim2 = dense_mat.shape
del binary_mat

## Random missing (RM)
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)
dense_tensor = dense_mat.reshape([dim1, 30, 24]).transpose(0, 2, 1)
sparse_tensor = sparse_mat.reshape([dim1, 30, 24]).transpose(0, 2, 1)
del dense_mat, sparse_mat

In [16]:
import time
start = time.time()
alpha = [1/3, 1/3, 1/3]
rho = 1e-4
epsilon = 1e-3
maxiter = 100
tensor_hat = HaLRTC_imputer(dense_tensor, sparse_tensor, alpha, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes'% ((end - start)/60.0))

Iter: 1
Tolerance: 0.193907
MAPE: 0.872952
RMSE: 21.3856

Iter: 2
Tolerance: 0.279604
MAPE: 0.692113
RMSE: 17.2804

Iter: 3
Tolerance: 0.280002
MAPE: 0.515869
RMSE: 13.2503

Iter: 4
Tolerance: 0.271091
MAPE: 0.35457
RMSE: 9.4631

Iter: 5
Tolerance: 0.258816
MAPE: 0.220113
RMSE: 6.13519

Iter: 6
Tolerance: 0.236083
MAPE: 0.144326
RMSE: 3.8922

Iter: 7
Tolerance: 0.190407
MAPE: 0.155552
RMSE: 3.57081

Iter: 8
Tolerance: 0.133529
MAPE: 0.192186
RMSE: 4.20487

Iter: 9
Tolerance: 0.0875637
MAPE: 0.2149
RMSE: 4.66671

Iter: 10
Tolerance: 0.0596708
MAPE: 0.217809
RMSE: 4.73512

Iter: 11
Tolerance: 0.0512944
MAPE: 0.204187
RMSE: 4.46345

Iter: 12
Tolerance: 0.0545674
MAPE: 0.179685
RMSE: 3.97088

Iter: 13
Tolerance: 0.0584223
MAPE: 0.15107
RMSE: 3.40138

Iter: 14
Tolerance: 0.0576311
MAPE: 0.125539
RMSE: 2.90027

Iter: 15
Tolerance: 0.0511644
MAPE: 0.108683
RMSE: 2.57027

Iter: 16
Tolerance: 0.0405338
MAPE: 0.101003
RMSE: 2.41901

Iter: 17
Tolerance: 0.0288431
MAPE: 0.0985892
RMSE: 2.37401

It

In [17]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.7

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]
dim1, dim2 = dense_mat.shape
del binary_mat

## Random missing (RM)
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)
dense_tensor = dense_mat.reshape([dim1, 30, 24]).transpose(0, 2, 1)
sparse_tensor = sparse_mat.reshape([dim1, 30, 24]).transpose(0, 2, 1)
del dense_mat, sparse_mat

pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
halrtc_lhat = dense_tensor[pos_test] - tensor_hat[pos_test]
np.save('halrtc_lhat.npy', halrtc_lhat)

In [18]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.3

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]
dim1, dim2 = dense_mat.shape
del binary_mat

## Non-random missing (NM)
dense_tensor = dense_mat.reshape([dim1, 30, 24]).transpose(0, 2, 1)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, 30) + 0.5 - missing_rate)[:, None, :]
del dense_mat

In [19]:
import time
start = time.time()
alpha = [1/3, 1/3, 1/3]
rho = 1e-4
epsilon = 1e-3
maxiter = 100
tensor_hat = HaLRTC_imputer(dense_tensor, sparse_tensor, alpha, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes'% ((end - start)/60.0))

Iter: 1
Tolerance: 0.142202
MAPE: 0.870634
RMSE: 21.2539

Iter: 2
Tolerance: 0.135213
MAPE: 0.743554
RMSE: 18.2126

Iter: 3
Tolerance: 0.114442
MAPE: 0.621529
RMSE: 15.2933

Iter: 4
Tolerance: 0.0976006
MAPE: 0.506032
RMSE: 12.5337

Iter: 5
Tolerance: 0.0875994
MAPE: 0.397342
RMSE: 9.93648

Iter: 6
Tolerance: 0.0813458
MAPE: 0.296708
RMSE: 7.51387

Iter: 7
Tolerance: 0.0755291
MAPE: 0.2081
RMSE: 5.32854

Iter: 8
Tolerance: 0.0676654
MAPE: 0.139692
RMSE: 3.5521

Iter: 9
Tolerance: 0.0558498
MAPE: 0.102811
RMSE: 2.51298

Iter: 10
Tolerance: 0.0383313
MAPE: 0.0985001
RMSE: 2.31715

Iter: 11
Tolerance: 0.0214282
MAPE: 0.103298
RMSE: 2.39276

Iter: 12
Tolerance: 0.0117829
MAPE: 0.103562
RMSE: 2.40112

Iter: 13
Tolerance: 0.00930069
MAPE: 0.100732
RMSE: 2.35044

Iter: 14
Tolerance: 0.0090215
MAPE: 0.0979536
RMSE: 2.30034

Iter: 15
Tolerance: 0.00838672
MAPE: 0.0963748
RMSE: 2.27422

Iter: 16
Tolerance: 0.00743267
MAPE: 0.095788
RMSE: 2.26766

Iter: 17
Tolerance: 0.00645672
MAPE: 0.0957008
RM

In [20]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.7

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]
dim1, dim2 = dense_mat.shape
del binary_mat

## Non-random missing (NM)
dense_tensor = dense_mat.reshape([dim1, 30, 24]).transpose(0, 2, 1)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, 30) + 0.5 - missing_rate)[:, None, :]
del dense_mat

In [21]:
import time
start = time.time()
alpha = [1/3, 1/3, 1/3]
rho = 1e-4
epsilon = 1e-3
maxiter = 100
tensor_hat = HaLRTC_imputer(dense_tensor, sparse_tensor, alpha, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes'% ((end - start)/60.0))

Iter: 1
Tolerance: 0.136749
MAPE: 0.952732
RMSE: 23.1671

Iter: 2
Tolerance: 0.134164
MAPE: 0.906701
RMSE: 22.0723

Iter: 3
Tolerance: 0.109504
MAPE: 0.862756
RMSE: 21.015

Iter: 4
Tolerance: 0.089506
MAPE: 0.821003
RMSE: 20.0008

Iter: 5
Tolerance: 0.0781584
MAPE: 0.780913
RMSE: 19.0241

Iter: 6
Tolerance: 0.0723827
MAPE: 0.741827
RMSE: 18.0778

Iter: 7
Tolerance: 0.069178
MAPE: 0.70328
RMSE: 17.1559

Iter: 8
Tolerance: 0.0669511
MAPE: 0.665047
RMSE: 16.2544

Iter: 9
Tolerance: 0.0650452
MAPE: 0.627046
RMSE: 15.3712

Iter: 10
Tolerance: 0.0633463
MAPE: 0.589312
RMSE: 14.5049

Iter: 11
Tolerance: 0.0618487
MAPE: 0.551956
RMSE: 13.6547

Iter: 12
Tolerance: 0.0605311
MAPE: 0.515136
RMSE: 12.821

Iter: 13
Tolerance: 0.0593046
MAPE: 0.479029
RMSE: 12.0047

Iter: 14
Tolerance: 0.0580936
MAPE: 0.443813
RMSE: 11.2069

Iter: 15
Tolerance: 0.0568743
MAPE: 0.409659
RMSE: 10.4285

Iter: 16
Tolerance: 0.0556134
MAPE: 0.376734
RMSE: 9.6706

Iter: 17
Tolerance: 0.0542759
MAPE: 0.345181
RMSE: 8.93463

### PeMS-4W

We generate **random missing (RM)** values on PeMS data set.

In [4]:
import numpy as np
import pandas as pd
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dense_tensor = mat2ten(dense_mat, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_tensor = np.random.rand(dense_mat.shape[0], 288, 4 * 7)

missing_rate = 0.3

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

In [5]:
import time
start = time.time()
alpha = [1/3, 1/3, 1/3]
rho = 1e-4
epsilon = 1e-3
maxiter = 100
tensor_hat = HaLRTC_imputer(dense_tensor, sparse_tensor, alpha, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes'% ((end - start)/60.0))

Iter: 1
Tolerance: 0.162504
MAPE: 0.747643
RMSE: 48.0704

Iter: 2
Tolerance: 0.17373
MAPE: 0.483965
RMSE: 31.5138

Iter: 3
Tolerance: 0.169525
MAPE: 0.234364
RMSE: 15.5622

Iter: 4
Tolerance: 0.147352
MAPE: 0.0494994
RMSE: 3.68009

Iter: 5
Tolerance: 0.0752858
MAPE: 0.115586
RMSE: 7.15935

Iter: 6
Tolerance: 0.0267006
MAPE: 0.153705
RMSE: 9.54833

Iter: 7
Tolerance: 0.00592663
MAPE: 0.146739
RMSE: 9.16161

Iter: 8
Tolerance: 0.0235193
MAPE: 0.110936
RMSE: 6.98242

Iter: 9
Tolerance: 0.0303539
MAPE: 0.0653842
RMSE: 4.25742

Iter: 10
Tolerance: 0.0261623
MAPE: 0.0296291
RMSE: 2.30527

Iter: 11
Tolerance: 0.0157296
MAPE: 0.0215302
RMSE: 1.85426

Iter: 12
Tolerance: 0.00570743
MAPE: 0.0233878
RMSE: 1.87473

Iter: 13
Tolerance: 0.0029137
MAPE: 0.0210664
RMSE: 1.75742

Iter: 14
Tolerance: 0.0043357
MAPE: 0.0185319
RMSE: 1.65999

Iter: 15
Tolerance: 0.00373095
MAPE: 0.0185547
RMSE: 1.66672

Iter: 16
Tolerance: 0.0022955
MAPE: 0.0195109
RMSE: 1.70836

Iter: 17
Tolerance: 0.00116862
MAPE: 0.019

In [6]:
import numpy as np
import pandas as pd
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dense_tensor = mat2ten(dense_mat, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_tensor = np.random.rand(dense_mat.shape[0], 288, 4 * 7)

missing_rate = 0.7

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

In [7]:
import time
start = time.time()
alpha = [1/3, 1/3, 1/3]
rho = 1e-4
epsilon = 1e-3
maxiter = 100
tensor_hat = HaLRTC_imputer(dense_tensor, sparse_tensor, alpha, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes'% ((end - start)/60.0))

Iter: 1
Tolerance: 0.161263
MAPE: 0.893708
RMSE: 57.2218

Iter: 2
Tolerance: 0.183983
MAPE: 0.778483
RMSE: 50.2084

Iter: 3
Tolerance: 0.181269
MAPE: 0.664861
RMSE: 43.3341

Iter: 4
Tolerance: 0.173952
MAPE: 0.555306
RMSE: 36.6806

Iter: 5
Tolerance: 0.165776
MAPE: 0.450946
RMSE: 30.2369

Iter: 6
Tolerance: 0.159123
MAPE: 0.352459
RMSE: 23.9702

Iter: 7
Tolerance: 0.154143
MAPE: 0.260377
RMSE: 17.8849

Iter: 8
Tolerance: 0.149356
MAPE: 0.17547
RMSE: 12.0812

Iter: 9
Tolerance: 0.142433
MAPE: 0.100744
RMSE: 6.94369

Iter: 10
Tolerance: 0.127589
MAPE: 0.0551887
RMSE: 4.16849

Iter: 11
Tolerance: 0.0913618
MAPE: 0.0840624
RMSE: 5.3829

Iter: 12
Tolerance: 0.0534457
MAPE: 0.112714
RMSE: 6.97769

Iter: 13
Tolerance: 0.0263943
MAPE: 0.125508
RMSE: 7.79449

Iter: 14
Tolerance: 0.00979536
MAPE: 0.124682
RMSE: 7.80296

Iter: 15
Tolerance: 0.0186167
MAPE: 0.112881
RMSE: 7.12912

Iter: 16
Tolerance: 0.0307053
MAPE: 0.0933398
RMSE: 5.97989

Iter: 17
Tolerance: 0.0378724
MAPE: 0.0699468
RMSE: 4.629

We generate **non-random missing (NM)** values on PeMS data set. Then, we conduct the imputation experiment.

In [12]:
import numpy as np
import pandas as pd
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dense_tensor = mat2ten(dense_mat, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_matrix = np.random.rand(dense_mat.shape[0], 4 * 7)

missing_rate = 0.3

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

In [13]:
import time
start = time.time()
alpha = [1/3, 1/3, 1/3]
rho = 1e-5
epsilon = 1e-3
maxiter = 100
tensor_hat = HaLRTC_imputer(dense_tensor, sparse_tensor, alpha, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes'% ((end - start)/60.0))

Iter: 1
Tolerance: 0.164378
MAPE: 0.743745
RMSE: 47.9493

Iter: 2
Tolerance: 0.154922
MAPE: 0.509454
RMSE: 33.1246

Iter: 3
Tolerance: 0.148226
MAPE: 0.294357
RMSE: 19.1232

Iter: 4
Tolerance: 0.137784
MAPE: 0.109656
RMSE: 7.37387

Iter: 5
Tolerance: 0.0953765
MAPE: 0.101664
RMSE: 6.64146

Iter: 6
Tolerance: 0.0369666
MAPE: 0.142417
RMSE: 8.83448

Iter: 7
Tolerance: 0.0200566
MAPE: 0.11932
RMSE: 7.77304

Iter: 8
Tolerance: 0.032733
MAPE: 0.077747
RMSE: 5.68817

Iter: 9
Tolerance: 0.0298398
MAPE: 0.0584402
RMSE: 4.64719

Iter: 10
Tolerance: 0.0192641
MAPE: 0.0624378
RMSE: 4.68247

Iter: 11
Tolerance: 0.00961385
MAPE: 0.0655479
RMSE: 4.78767

Iter: 12
Tolerance: 0.00607767
MAPE: 0.0627286
RMSE: 4.68839

Iter: 13
Tolerance: 0.00627358
MAPE: 0.0579073
RMSE: 4.50989

Iter: 14
Tolerance: 0.00568861
MAPE: 0.0541945
RMSE: 4.37249

Iter: 15
Tolerance: 0.00436141
MAPE: 0.0522923
RMSE: 4.29989

Iter: 16
Tolerance: 0.00305355
MAPE: 0.0516487
RMSE: 4.26908

Iter: 17
Tolerance: 0.00241376
MAPE: 0.05

In [4]:
import numpy as np
import pandas as pd
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dense_tensor = mat2ten(dense_mat, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_matrix = np.random.rand(dense_mat.shape[0], 4 * 7)

missing_rate = 0.7

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

In [5]:
import time
start = time.time()
alpha = [1/3, 1/3, 1/3]
rho = 1e-5
epsilon = 1e-3
maxiter = 100
tensor_hat = HaLRTC_imputer(dense_tensor, sparse_tensor, alpha, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes'% ((end - start)/60.0))

Iter: 1
Tolerance: 0.140875
MAPE: 0.908662
RMSE: 58.1625

Iter: 2
Tolerance: 0.130072
MAPE: 0.823523
RMSE: 53.0064

Iter: 3
Tolerance: 0.124245
MAPE: 0.742864
RMSE: 48.0815

Iter: 4
Tolerance: 0.119596
MAPE: 0.666269
RMSE: 43.3382

Iter: 5
Tolerance: 0.115362
MAPE: 0.592803
RMSE: 38.765

Iter: 6
Tolerance: 0.111279
MAPE: 0.522216
RMSE: 34.3638

Iter: 7
Tolerance: 0.107255
MAPE: 0.454588
RMSE: 30.1363

Iter: 8
Tolerance: 0.103335
MAPE: 0.390024
RMSE: 26.0837

Iter: 9
Tolerance: 0.0994727
MAPE: 0.328667
RMSE: 22.2146

Iter: 10
Tolerance: 0.0954309
MAPE: 0.270908
RMSE: 18.5533

Iter: 11
Tolerance: 0.0910948
MAPE: 0.217234
RMSE: 15.1371

Iter: 12
Tolerance: 0.0862522
MAPE: 0.16847
RMSE: 12.0328

Iter: 13
Tolerance: 0.0806221
MAPE: 0.126046
RMSE: 9.35807

Iter: 14
Tolerance: 0.0738548
MAPE: 0.0930864
RMSE: 7.30359

Iter: 15
Tolerance: 0.0652308
MAPE: 0.07599
RMSE: 6.096

Iter: 16
Tolerance: 0.053561
MAPE: 0.075247
RMSE: 5.74521

Iter: 17
Tolerance: 0.0400716
MAPE: 0.0805153
RMSE: 5.85041

I

### PeMS-8W

We generate **random missing (RM)** values on PeMS data set.

In [4]:
import numpy as np
import pandas as pd
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 9):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dense_tensor = mat2ten(dense_mat, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_tensor = np.random.rand(dense_mat.shape[0], 288, 8 * 7)

missing_rate = 0.3

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

In [5]:
import time
start = time.time()
alpha = [1/3, 1/3, 1/3]
rho = 1e-4
epsilon = 1e-3
maxiter = 100
tensor_hat = HaLRTC_imputer(dense_tensor, sparse_tensor, alpha, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes'% ((end - start)/60.0))

Iter: 1
Tolerance: 0.14037
MAPE: 0.782328
RMSE: 50.3169

Iter: 2
Tolerance: 0.147164
MAPE: 0.558324
RMSE: 36.3305

Iter: 3
Tolerance: 0.144577
MAPE: 0.343187
RMSE: 22.706

Iter: 4
Tolerance: 0.136253
MAPE: 0.148877
RMSE: 10.215

Iter: 5
Tolerance: 0.106025
MAPE: 0.0515522
RMSE: 3.75906

Iter: 6
Tolerance: 0.0525445
MAPE: 0.111476
RMSE: 6.94239

Iter: 7
Tolerance: 0.0195182
MAPE: 0.137761
RMSE: 8.5794

Iter: 8
Tolerance: 0.0075469
MAPE: 0.127863
RMSE: 8.01016

Iter: 9
Tolerance: 0.0213228
MAPE: 0.095586
RMSE: 6.05651

Iter: 10
Tolerance: 0.0258518
MAPE: 0.0573138
RMSE: 3.79287

Iter: 11
Tolerance: 0.0215011
MAPE: 0.0289175
RMSE: 2.27052

Iter: 12
Tolerance: 0.0126488
MAPE: 0.021566
RMSE: 1.87344

Iter: 13
Tolerance: 0.00467374
MAPE: 0.0223974
RMSE: 1.85415

Iter: 14
Tolerance: 0.00198838
MAPE: 0.0213857
RMSE: 1.80604

Iter: 15
Tolerance: 0.0027865
MAPE: 0.0202272
RMSE: 1.77159

Iter: 16
Tolerance: 0.00244806
MAPE: 0.0200951
RMSE: 1.77803

Iter: 17
Tolerance: 0.00162505
MAPE: 0.0204431
R

In [6]:
import numpy as np
import pandas as pd
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 9):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dense_tensor = mat2ten(dense_mat, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_tensor = np.random.rand(dense_mat.shape[0], 288, 8 * 7)

missing_rate = 0.7

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

In [7]:
import time
start = time.time()
alpha = [1/3, 1/3, 1/3]
rho = 1e-4
epsilon = 1e-3
maxiter = 100
tensor_hat = HaLRTC_imputer(dense_tensor, sparse_tensor, alpha, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes'% ((end - start)/60.0))

Iter: 1
Tolerance: 0.142141
MAPE: 0.907855
RMSE: 58.171

Iter: 2
Tolerance: 0.15666
MAPE: 0.810745
RMSE: 52.2869

Iter: 3
Tolerance: 0.153815
MAPE: 0.715069
RMSE: 46.5338

Iter: 4
Tolerance: 0.147833
MAPE: 0.622377
RMSE: 40.9517

Iter: 5
Tolerance: 0.140998
MAPE: 0.533399
RMSE: 35.5228

Iter: 6
Tolerance: 0.13542
MAPE: 0.448423
RMSE: 30.2115

Iter: 7
Tolerance: 0.131523
MAPE: 0.367543
RMSE: 24.9987

Iter: 8
Tolerance: 0.128453
MAPE: 0.290795
RMSE: 19.8968

Iter: 9
Tolerance: 0.125265
MAPE: 0.218332
RMSE: 14.9552

Iter: 10
Tolerance: 0.121183
MAPE: 0.150849
RMSE: 10.2937

Iter: 11
Tolerance: 0.114775
MAPE: 0.0910389
RMSE: 6.26643

Iter: 12
Tolerance: 0.101607
MAPE: 0.0530583
RMSE: 4.07487

Iter: 13
Tolerance: 0.0738344
MAPE: 0.0698544
RMSE: 4.67347

Iter: 14
Tolerance: 0.0453289
MAPE: 0.0927894
RMSE: 5.85756

Iter: 15
Tolerance: 0.0240325
MAPE: 0.104522
RMSE: 6.5638

Iter: 16
Tolerance: 0.0091305
MAPE: 0.1056
RMSE: 6.66208

Iter: 17
Tolerance: 0.0131239
MAPE: 0.0978429
RMSE: 6.22478

It

We generate **non-random missing (NM)** values on PeMS data set. Then, we conduct the imputation experiment.

In [8]:
import numpy as np
import pandas as pd
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 9):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dense_tensor = mat2ten(dense_mat, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_matrix = np.random.rand(dense_mat.shape[0], 8 * 7)

missing_rate = 0.3

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

In [9]:
import time
start = time.time()
alpha = [1/3, 1/3, 1/3]
rho = 1e-5
epsilon = 1e-3
maxiter = 100
tensor_hat = HaLRTC_imputer(dense_tensor, sparse_tensor, alpha, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes'% ((end - start)/60.0))

Iter: 1
Tolerance: 0.173341
MAPE: 0.728282
RMSE: 47.0542

Iter: 2
Tolerance: 0.166331
MAPE: 0.47679
RMSE: 31.0319

Iter: 3
Tolerance: 0.160301
MAPE: 0.245805
RMSE: 15.8881

Iter: 4
Tolerance: 0.146767
MAPE: 0.0691888
RMSE: 5.20334

Iter: 5
Tolerance: 0.0817357
MAPE: 0.140975
RMSE: 8.66627

Iter: 6
Tolerance: 0.0188958
MAPE: 0.153648
RMSE: 9.49318

Iter: 7
Tolerance: 0.0317748
MAPE: 0.107761
RMSE: 7.09369

Iter: 8
Tolerance: 0.0381911
MAPE: 0.0621565
RMSE: 4.86471

Iter: 9
Tolerance: 0.0276539
MAPE: 0.0566616
RMSE: 4.49975

Iter: 10
Tolerance: 0.0134418
MAPE: 0.0645257
RMSE: 4.74767

Iter: 11
Tolerance: 0.00582363
MAPE: 0.0631845
RMSE: 4.69801

Iter: 12
Tolerance: 0.00707797
MAPE: 0.0570332
RMSE: 4.46976

Iter: 13
Tolerance: 0.00711673
MAPE: 0.0517809
RMSE: 4.28704

Iter: 14
Tolerance: 0.00530746
MAPE: 0.0492647
RMSE: 4.19975

Iter: 15
Tolerance: 0.00339755
MAPE: 0.0484796
RMSE: 4.16335

Iter: 16
Tolerance: 0.00252274
MAPE: 0.0483315
RMSE: 4.14365

Iter: 17
Tolerance: 0.00238577
MAPE: 0

In [10]:
import numpy as np
import pandas as pd
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 9):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dense_tensor = mat2ten(dense_mat, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_matrix = np.random.rand(dense_mat.shape[0], 8 * 7)

missing_rate = 0.7

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

In [11]:
import time
start = time.time()
alpha = [1/3, 1/3, 1/3]
rho = 1e-5
epsilon = 1e-3
maxiter = 100
tensor_hat = HaLRTC_imputer(dense_tensor, sparse_tensor, alpha, rho, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes'% ((end - start)/60.0))

Iter: 1
Tolerance: 0.155267
MAPE: 0.897015
RMSE: 57.5207

Iter: 2
Tolerance: 0.147038
MAPE: 0.799105
RMSE: 51.5622

Iter: 3
Tolerance: 0.143054
MAPE: 0.705754
RMSE: 45.7587

Iter: 4
Tolerance: 0.139714
MAPE: 0.615809
RMSE: 40.0954

Iter: 5
Tolerance: 0.136429
MAPE: 0.528805
RMSE: 34.5807

Iter: 6
Tolerance: 0.133176
MAPE: 0.444718
RMSE: 29.2187

Iter: 7
Tolerance: 0.129896
MAPE: 0.36377
RMSE: 24.0263

Iter: 8
Tolerance: 0.126385
MAPE: 0.286361
RMSE: 19.0374

Iter: 9
Tolerance: 0.1224
MAPE: 0.213221
RMSE: 14.326

Iter: 10
Tolerance: 0.117447
MAPE: 0.146271
RMSE: 10.0764

Iter: 11
Tolerance: 0.110765
MAPE: 0.0918669
RMSE: 6.7714

Iter: 12
Tolerance: 0.0991854
MAPE: 0.0730182
RMSE: 5.47106

Iter: 13
Tolerance: 0.0776696
MAPE: 0.092264
RMSE: 6.22371

Iter: 14
Tolerance: 0.0510863
MAPE: 0.112022
RMSE: 7.23604

Iter: 15
Tolerance: 0.030048
MAPE: 0.116228
RMSE: 7.58547

Iter: 16
Tolerance: 0.0280783
MAPE: 0.106261
RMSE: 7.22328

Iter: 17
Tolerance: 0.0369164
MAPE: 0.0893296
RMSE: 6.44921

Ite

### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>